In [26]:
import numpy as np
from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import mutual_info_regression

# Generate artificial dataset
X, y = make_friedman1(n_samples=1000, n_features=10, noise=0.1, random_state=42)
y = y.ravel()  # Flatten the target variable to 1D

# Create a list of significant features (known ground truth)
significant_features = [0, 2, 4, 6, 8]

# Define a stopping rule
def stopping_rule(selected_features, jmi_scores, threshold=0.01):
    if len(selected_features) == 0:
        return False
    else:
        return np.max(jmi_scores) < threshold

def jmi(X, y, selected_features):
    """
    Joint Mutual Information (JMI) criterion for feature selection with continuous features.

    Args:
        X (numpy.ndarray): Feature matrix.
        y (numpy.ndarray): Target variable.
        selected_features (list): List of indices of already selected features.

    Returns:
        numpy.ndarray: JMI scores for each remaining feature.
    """
    jmi_scores = []
    for i in range(X.shape[1]):
        if i not in selected_features:
            # Calculate I(S;Y)
            i_s_y = sum(mutual_info_regression(X[:, j].reshape(-1, 1), y) for j in selected_features)

            # Calculate I(X_i;Y)
            i_xi_y = mutual_info_regression(X[:, i].reshape(-1, 1), y)

            # Calculate I(X_i;S)
            i_xi_s = sum(mutual_info_regression(X[:, i].reshape(-1, 1), X[:, j].reshape(-1, 1)) for j in selected_features)

            # Calculate I(X_i;S|Y)
            i_xi_s_y = sum(
                mutual_info_regression(np.column_stack([X[:, i], X[:, j]]), y) - i_xi_y - mutual_info_regression(X[:, j].reshape(-1, 1), y)
                for j in selected_features
            )

            # Calculate JMI
            jmi_score = i_s_y + i_xi_y - i_xi_s + i_xi_s_y
            jmi_scores.append(jmi_score)

    return np.array(jmi_scores)

# Select the feature with maximum mutual information with y
mi_scores = [mutual_info_regression(X[:, i].reshape(-1, 1), y) for i in range(X.shape[1])]
first_feature = np.argmax(mi_scores)
selected_features = [first_feature]

# Perform feature selection using JMI
while True:
    jmi_scores = jmi(X, y, selected_features)
    best_feature_idx = np.argmax(jmi_scores)
    selected_features.append(best_feature_idx)
    if stopping_rule(selected_features, jmi_scores):
        break

# Evaluate the selected features
print(f"Selected features: {sorted(selected_features)}")
print(f"Significant features: {sorted(significant_features)}")

# Train a model with the selected features
X_selected = X[:, selected_features]
model = LinearRegression()
model.fit(X_selected, y)
y_pred = model.predict(X_selected)
score = r2_score(y, y_pred)
print(f"R-squared score: {score}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

IndexError: index 11 is out of bounds for axis 1 with size 10

I have dificulties implemnting JMI